In [4]:
from faker import Faker
import pandas as pd 
import numpy as np

In [12]:
faker = Faker()
faker.name()

t = []
for n in range(7):
    t.append(faker.name())

test =   ['Desiree Oconnor',
 'Taylor Walker',
 'Sharon Woods',
 'Amanda Hernandez',
 'Karen Thompson',
 'Christopher Williams',
 'David Oliver']

test_nest = [
[1,['Desiree Oconnor','Desiree Oconnor','Desiree Oconnor','Desiree Oconnor','Desiree Oconnor']],
[2,['Taylor Walker','Taylor Walker','Taylor Walker','Desiree Oconnor']],
[3,['Amanda Hernandez','Amanda Hernandez','Amanda Hernandez','Amanda Hernandez','Desiree Oconnor']],
    ### there will be blanks within the dataset or abstract list
[4,[]]]

testing_loop = pd.DataFrame(list(test_nest),columns = ['Index_number','Abstract_trim_2'])

#for id,i in enumerate(test_nest):
#   testing_loop.values[id] = i 

In [13]:
from collections import Counter
import numpy as np
testing_loop

def getUniqueWords(allWords):
    uniqueWords = [] 
    for i in allWords:
        if not i in uniqueWords:
            uniqueWords.append(i)
    return uniqueWords

testing_loop_long = testing_loop
testing_loop_long['Abstract_trim_2_distinct'] = testing_loop_long['Abstract_trim_2']
for id,i in enumerate(testing_loop_long['Abstract_trim_2']):
    testing_loop_long['Abstract_trim_2_distinct'].iloc[id] = getUniqueWords(i)
        
testing_loop_long = testing_loop_long[testing_loop_long.astype(str)['Abstract_trim_2_distinct'] != '[]']
testing_loop = testing_loop[testing_loop.astype(str)['Abstract_trim_2_distinct'] != '[]']    


###### Using the Counter Packages to select the most counted words and it's count frequency
testing_loop['abstract_counts'] = [Counter(x).most_common() for x in testing_loop['Abstract_trim_2']]
#pd.DataFrame(testing_loop['abstract_counts'], columns=['Word','Count'])
#pd.DataFrame(testing_loop['abstract_counts'])
#pd.DataFrame(list(map(None,testing_loop['abstract_counts'])))
### Interesting Unnesting a Lists
### A list comprehensions

### Remove any empty strings which will remove that row 
testing_loop = testing_loop[testing_loop.astype(str)['abstract_counts'] != '[]']

testing_loop_long = testing_loop[['Index_number','Abstract_trim_2_distinct','Abstract_trim_2']].explode('Abstract_trim_2_distinct')

testing_loop_long['abstract_counts_unlist'] = [val for sublist in testing_loop['abstract_counts'] for val in sublist]

#pd.DataFrame(testing_loop['abstract_counts_unlist'])

#### Unzip the individual tuples and covert into seperate columns
#testing_loop_long['Word'] = testing_loop_long['Abstract_trim_2_distinct']
#testing_loop_long['Count'] = testing_loop_long['Abstract_trim_2_distinct']
testing_loop_long['Word'], testing_loop_long['Count'] = zip(*testing_loop_long['abstract_counts_unlist'])
testing_loop_long['Binary_count'] = 1

testing_loop_long["Total_words_in_abstract"] = testing_loop_long['Abstract_trim_2']
for id,i in enumerate(testing_loop_long['Abstract_trim_2']):
    testing_loop_long["Total_words_in_abstract"].iloc[id] = len(testing_loop_long['Abstract_trim_2'].iloc[id])
testing_loop_long['Cal_Term_Freq_TF'] =  (testing_loop_long['Count']/testing_loop_long["Total_words_in_abstract"])
    
testing_loop_long_Count = testing_loop_long[['Index_number','Word','Count']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Count').fillna(0).add_prefix('Count_')   
testing_loop_long_Count = pd.DataFrame(testing_loop_long_Count.to_records())

testing_loop_long_TF = testing_loop_long[['Index_number','Word','Cal_Term_Freq_TF']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Cal_Term_Freq_TF',aggfunc = np.sum).fillna(0).add_prefix('TF_')      
testing_loop_long_TF = pd.DataFrame(testing_loop_long_TF.to_records()) 

testing_loop_long_Binary = testing_loop_long[['Index_number','Word','Binary_count']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Binary_count').fillna(0).add_prefix('Binary_') 
testing_loop_long_Binary = pd.DataFrame(testing_loop_long_Binary.to_records()) 

testing_loop_long['abstract_concat_list'] = testing_loop_long['Abstract_trim_2']
for i in range(len(testing_loop_long['Abstract_trim_2'])):
    testing_loop_long['abstract_concat_list'].values[i] = list(testing_loop['Abstract_trim_2'])
    
testing_loop_long['dsad'] = testing_loop_long['abstract_concat_list']
for id1,o in enumerate(testing_loop_long['abstract_concat_list']):
    for id2,i in enumerate(o):
        testing_loop_long['dsad'].iloc[id1][id2] = [i for i in testing_loop_long['abstract_concat_list'].iloc[id1][id2] if i == testing_loop_long['Word'].values[id1]] 

for id,i in enumerate(testing_loop_long['dsad']):
    testing_loop_long['dsad'].values[id] = [x for x in testing_loop_long['dsad'].values[id] if x != []]        

testing_loop_long['times_mentioned_in_abstracts'] = [len(x) for x in testing_loop_long['dsad']]   
testing_loop_long['Total_number_abstracts'] = len(testing_loop["Abstract_trim_2"])       
testing_loop_long['Cal_Inverse_Document_Freq_IDF'] = np.log10(testing_loop_long['Total_number_abstracts']/testing_loop_long['times_mentioned_in_abstracts'])

testing_loop_long_IDF = testing_loop_long[['Index_number','Word','Cal_Inverse_Document_Freq_IDF']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Cal_Inverse_Document_Freq_IDF').fillna(0).add_prefix('IDF_')  
testing_loop_long_IDF = pd.DataFrame(testing_loop_long_IDF.to_records()) 

testing_loop_wide = testing_loop_long_IDF.set_index('Index_number').join(testing_loop_long_TF.set_index('Index_number')).join(testing_loop_long_Count.set_index('Index_number')).join(testing_loop_long_Binary.set_index('Index_number'))
testing_loop_wide.to_csv(r'C:\Users\andym\Desktop\Datafancy\Journal_managementNP\03-test_datasets\testing_loop_wide_2.csv')

np.log(3)
### Now add the word count Ratio frequency
#long_dataframe_unique = pd.DataFrame(long_dataframe_unique.to_records())
#long_dataframe_unique.columns = [hdr.replace("('binary_count',","Binary_count_").replace(")", "").replace("'","").replace("'","").replace("_ ","_").replace(" ","_") \
#  for hdr in long_dataframe_unique.columns]

C:\Users\andym\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


PermissionError: [Errno 13] Permission denied: 'C:\\Users\\andym\\Desktop\\Datafancy\\Journal_managementNP\\03-test_datasets\\testing_loop_wide_2.csv'

In [7]:
def word_count_df(testing_loop):

    from collections import Counter
    import numpy as np
    testing_loop

    def getUniqueWords(allWords):
        uniqueWords = [] 
        for i in allWords:
            if not i in uniqueWords:
                uniqueWords.append(i)
        return uniqueWords

    testing_loop_long = testing_loop
    testing_loop_long['Abstract_trim_2_distinct'] = testing_loop_long['Abstract_trim_2']
    for id,i in enumerate(testing_loop_long['Abstract_trim_2']):
        testing_loop_long['Abstract_trim_2_distinct'].iloc[id] = getUniqueWords(i)
        
    testing_loop_long = testing_loop_long[testing_loop_long.astype(str)['Abstract_trim_2_distinct'] != '[]']
    testing_loop = testing_loop[testing_loop.astype(str)['Abstract_trim_2_distinct'] != '[]']    


###### Using the Counter Packages to select the most counted words and it's count frequency
    testing_loop['abstract_counts'] = [Counter(x).most_common() for x in testing_loop['Abstract_trim_2']]
#pd.DataFrame(testing_loop['abstract_counts'], columns=['Word','Count'])
#pd.DataFrame(testing_loop['abstract_counts'])
#pd.DataFrame(list(map(None,testing_loop['abstract_counts'])))
### Interesting Unnesting a Lists
### A list comprehensions

### Remove any empty strings which will remove that row 
    testing_loop = testing_loop[testing_loop.astype(str)['abstract_counts'] != '[]']

    testing_loop_long = testing_loop[['Index_number','Abstract_trim_2_distinct','Abstract_trim_2']].explode('Abstract_trim_2_distinct')

    testing_loop_long['abstract_counts_unlist'] = [val for sublist in testing_loop['abstract_counts'] for val in sublist]

#pd.DataFrame(testing_loop['abstract_counts_unlist'])

#### Unzip the individual tuples and covert into seperate columns
#testing_loop_long['Word'] = testing_loop_long['Abstract_trim_2_distinct']
#testing_loop_long['Count'] = testing_loop_long['Abstract_trim_2_distinct']
    testing_loop_long['Word'], testing_loop_long['Count'] = zip(*testing_loop_long['abstract_counts_unlist'])
    testing_loop_long['Binary_count'] = 1

    testing_loop_long["Total_words_in_abstract"] = testing_loop_long['Abstract_trim_2']
    for id,i in enumerate(testing_loop_long['Abstract_trim_2']):
        testing_loop_long["Total_words_in_abstract"].iloc[id] = len(testing_loop_long['Abstract_trim_2'].iloc[id])
    testing_loop_long['Cal_Term_Freq_TF'] =  (testing_loop_long['Count']/testing_loop_long["Total_words_in_abstract"])
    
    testing_loop_long_Count = testing_loop_long[['Index_number','Word','Count']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Count').fillna(0).add_prefix('Count_')   
    testing_loop_long_Count = pd.DataFrame(testing_loop_long_Count.to_records())

    testing_loop_long_TF = testing_loop_long[['Index_number','Word','Cal_Term_Freq_TF']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Cal_Term_Freq_TF',aggfunc = np.sum).fillna(0).add_prefix('TF_')      
    testing_loop_long_TF = pd.DataFrame(testing_loop_long_TF.to_records()) 

    testing_loop_long_Binary = testing_loop_long[['Index_number','Word','Binary_count']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Binary_count').fillna(0).add_prefix('Binary_') 
    testing_loop_long_Binary = pd.DataFrame(testing_loop_long_Binary.to_records()) 

    testing_loop_long['abstract_concat_list'] = testing_loop_long['Abstract_trim_2']
    for i in range(len(testing_loop_long['Abstract_trim_2'])):
        testing_loop_long['abstract_concat_list'].values[i] = list(testing_loop['Abstract_trim_2'])
    
    testing_loop_long['dsad'] = testing_loop_long['abstract_concat_list']
    for id1,o in enumerate(testing_loop_long['abstract_concat_list']):
        for id2,i in enumerate(o):
            testing_loop_long['dsad'].iloc[id1][id2] = [i for i in testing_loop_long['abstract_concat_list'].iloc[id1][id2] if i == testing_loop_long['Word'].values[id1]] 

    for id,i in enumerate(testing_loop_long['dsad']):
        testing_loop_long['dsad'].values[id] = [x for x in testing_loop_long['dsad'].values[id] if x != []]        

    testing_loop_long['times_mentioned_in_abstracts'] = [len(x) for x in testing_loop_long['dsad']]   
    testing_loop_long['Total_number_abstracts'] = len(testing_loop["Abstract_trim_2"])       
    testing_loop_long['Cal_Inverse_Document_Freq_IDF'] = np.log10(testing_loop_long['Total_number_abstracts']/testing_loop_long['times_mentioned_in_abstracts'])

    testing_loop_long_IDF = testing_loop_long[['Index_number','Word','Cal_Inverse_Document_Freq_IDF']].pivot_table(index = 'Index_number',columns = 'Word' , values = 'Cal_Inverse_Document_Freq_IDF').fillna(0).add_prefix('IDF_')  
    testing_loop_long_IDF = pd.DataFrame(testing_loop_long_IDF.to_records()) 

    testing_loop_wide = testing_loop_long_IDF.set_index('Index_number').join(testing_loop_long_TF.set_index('Index_number')).join(testing_loop_long_Count.set_index('Index_number')).join(testing_loop_long_Binary.set_index('Index_number'))

    return testing_loop_wide

### Now add the word count Ratio frequency
#long_dataframe_unique = pd.DataFrame(long_dataframe_unique.to_records())
#long_dataframe_unique.columns = [hdr.replace("('binary_count',","Binary_count_").replace(")", "").replace("'","").replace("'","").replace("_ ","_").replace(" ","_") \
#  for hdr in long_dataframe_unique.columns]

In [8]:
word_count_df(testing_loop = testing_loop)

C:\Users\andym\Anaconda3\lib\site-packages\pandas\core\indexing.py:1636: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\andym\Anaconda3\lib\site-packages\ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,IDF_Amanda Hernandez,IDF_Desiree Oconnor,IDF_Taylor Walker,TF_Amanda Hernandez,TF_Desiree Oconnor,TF_Taylor Walker,Count_Amanda Hernandez,Count_Desiree Oconnor,Count_Taylor Walker,Binary_Amanda Hernandez,Binary_Desiree Oconnor,Binary_Taylor Walker
Index_number,,,,,,,,,,,,
1,0.000000,0.0,0.000000,0.0,1.00,0.00,0.0,5.0,0.0,0.0,1.0,0.0
2,0.000000,0.0,1.098612,0.0,0.25,0.75,0.0,1.0,3.0,0.0,1.0,1.0
3,1.098612,0.0,0.000000,0.8,0.20,0.00,4.0,1.0,0.0,1.0,1.0,0.0
